In [2]:
import json
import os
import pathlib
import string
import typing


In [3]:
def create_bank_from_sources() -> None:
    TARGET_SIZE = 5

    def filter_words(words: typing.Iterable[str]) -> typing.Iterable[str]:
        for word in words:
            if len(word) != TARGET_SIZE:
                continue
            if any(c not in string.ascii_lowercase for c in word):
                continue
            yield word

    def load_words_from_json(path: str) -> typing.Iterable[str]:
        return filter_words(json.load(open(path, 'r')))

    def load_words_from_txt(path: str) -> typing.Iterable[str]:
        lines = open(path, 'r').readlines()
        return filter_words(map(str.strip, lines))

    co_wordle_words = set(
        load_words_from_json('./bank_source/co_wordle_5_letter_guesses.json'))
    co_wordle_raw_targets = set(
        load_words_from_json('./bank_source/dordle_wordle_targets.json'))
    co_wordle_targets = co_wordle_raw_targets & co_wordle_words
    wordle_unlimited_targets = set(
        load_words_from_json('./bank_source/wordle_unlimited_targets.json'))
    wordle_unlimited_words = wordle_unlimited_targets | set(
        load_words_from_json('./bank_source/wordle_unlimited_guesses.json'))
    words = co_wordle_words | wordle_unlimited_words
    targets = co_wordle_targets | wordle_unlimited_targets
    guesses = words - targets

    common_words = set(
        list(load_words_from_txt('./bank_source/cspell_en_gb_dict.txt')) +
        list(load_words_from_txt('./bank_source/cspell_en_us_dict.txt')))
    common_guesses = guesses & common_words
    uncommon_guesses = guesses - common_guesses

    BANK_PATH = pathlib.Path('../../bank/co_wordle_unlimited')
    os.makedirs(BANK_PATH, exist_ok=True)

    def dump_words_to_txt(
        words: typing.Iterable[str],
        path: pathlib.Path,
    ) -> None:
        lines = sorted(w + '\n' for w in words)
        open(path, 'w').writelines(lines)

    dump_words_to_txt(uncommon_guesses, BANK_PATH / 'uncommon_guesses.txt')
    dump_words_to_txt(common_guesses, BANK_PATH / 'common_guesses.txt')
    dump_words_to_txt(targets, BANK_PATH / 'targets.txt')

    print(f'{len(uncommon_guesses)=} {len(common_guesses)=} {len(targets)=}')


create_bank_from_sources()


len(uncommon_guesses)=7193 len(common_guesses)=3409 len(targets)=2394
